In [1]:
import json
from pprint import pprint
Settings = json.load(open('settings.txt'))
pprint(Settings)
import sys
sys.path.insert(0,'../')
from numpy import genfromtxt
import numpy as np
from pak.evaluation import MOTA
from cabbage.data.video import VideoData
from experiments import MOT16_Experiments
from time import time

root = Settings['data_root']

mot16 = MOT16_Experiments(root)

print("check:")
print(mot16.mot16_11_detections.shape)
print("")


Hy = VideoData(mot16.get_MOT16_11_trajectories())  # true detections
Hy = Hy.get_n_first_frames(100)
print(Hy.shape)


__start = time()
result = genfromtxt('../build/RESULT.txt', delimiter=' ', dtype='uint32')
__end = time()
print('elapsed', __end - __start)
print('load results:', result.shape)



def extract_ids(result):
    #TODO: make this stuff better..
    
    candidates = []
    n = np.max(result[:,1])  # total number of nodes
    
    Nodes = [i for i in range(n+1)]
    
    
    for i, j, edge in result:
        assert(i < j)
        has_edge = edge == 0
        
        if has_edge:
            repres = Nodes[i]
            Nodes[j] = repres

    #print(Nodes)     
    return Nodes

__start = time()
Nodes = extract_ids(result)
__end = time()
print('elapsed', __end - __start)
print('finished extracting ids')

# translate to sensible id's
n_unique = len(np.unique(Nodes))
lookup = {}
for new,old in zip(list(range(n_unique)), np.unique(Nodes)):
    lookup[old] = new

for i in range(len(Nodes)):
    Nodes[i] = lookup[Nodes[i]]

    
print("Unique ids:", n_unique)

{'data_root': '/home/data',
 'deepmatch': '/home/deepmatching_1.2.2_c++/deepmatching-static',
 'graph_solver': '/home/cabbage/build/sample'}
check:
(8125, 6)

(501, 6)


KeyboardInterrupt: 

In [ ]:
from pak.datasets.MOT import MOT16
mot16__ = MOT16(root)
X, Y_det, Y_gt = mot16__.get_train("MOT16-11", memmapped=True)
print(Y_det[0:6].astype('object'))

print(mot16.mot16_11_detections.shape)

In [ ]:
from pak.post_processing import MOT
ALL_DETECTIONS = mot16.mot16_11_detections[0:len(Nodes)]  # as we do not need all frames..

print(ALL_DETECTIONS[0:6])

Dt = []
for i, (pid, data) in enumerate(zip(Nodes, ALL_DETECTIONS)):
    frame, x, y, w, h, score = data
    Dt.append((frame, pid, x, y, w, h, score))
Dt = np.array(Dt)

print(Dt[0:6])

Dt = MOT.remove_short_tracks(
    MOT.remove_duplicates(Dt, lambda x: x[5]),
    min_length=1)


Dt = Dt[:,0:6]
print(Dt.shape)

Gt = VideoData(mot16.get_MOT16_11_gt_trajectories()).get_n_first_frames(237)

print(Gt.shape)

threshold=0.5
result, info = MOTA.evaluate_aabb(Gt, Dt, threshold, info=True)


print("MOTA:", result)
print("\tinfo:", info)
